In [1]:
import os

In [2]:
os.environ.setdefault('LIBROSA_CACHE_DIR', '/tmp/librosa/')

'/tmp/librosa/'

In [3]:
import librosa

In [4]:
import mir_eval

In [5]:
from collections import OrderedDict

In [6]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=3)
pd.set_option('precision', 4, "display.max_rows", 999)

In [7]:
def make_onset_corpus(iso_path):
    
    # Beat files
    annotations = librosa.util.find_files(beat_path, ext='wav')
    
    audio = [ann.replace('.wav', '.csv') for ann in annotations]
    
    data = []
    for aud, ann in zip(audio, annotations):
        if os.path.exists(aud) and os.path.exists(ann):
            data.append((aud, ann))
    
    return pd.DataFrame(data=data, columns=['audio', 'annotation'])

In [ ]:
    #kwargs.setdefault('pre_max', 0.03*sr//hop_length)       # 30ms
    #kwargs.setdefault('post_max', 0.00*sr//hop_length + 1)  # 0ms
    #kwargs.setdefault('pre_avg', 0.10*sr//hop_length)       # 100ms
    #kwargs.setdefault('post_avg', 0.10*sr//hop_length + 1)  # 100ms
    #kwargs.setdefault('wait', 0.03*sr//hop_length)          # 30ms
    #kwargs.setdefault('delta', 0.06)

In [9]:
def analyze(dframe):
    
    index = dframe.index[0]
    
    print 'Processing {}'.format(os.path.basename(dframe['audio'][index]))
    
    # Load the truth
    ref_times = pd.read_table(dframe['annotation'][index], header=None, sep='\s+')[0].values

    # Load the audio
    y, sr = librosa.load(dframe['audio'][index])
    
    # Construct the output container
    results = []
    hop_length = 512
    
    effective_sr = sr // hop_length
    
    # Onset strength parameters
    for aggregate in [np.mean, np.median]:
        for fmax in [8000, None]:
            for n_mels in [32, 64, 128]:
                # Compute the onset detection function
                oenv = librosa.onset.onset_strength(y=y,
                                                    sr=sr,
                                                    aggregate=aggregate,
                                                    fmax=fmax,
                                                    n_mels=n_mels)
                
                for pre_max in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06]:
                    for post_max in [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06]:
                onsets = librosa.onset.onset_detect(onset_envelope=oenv, sr=sr)
                # Tempo estimator parameters
                for ac_size in [2, 4, 8, 16]:
                    for std_bpm in [0.25, 0.5, 1.0, 2.0]:
                        tempo = librosa.beat.estimate_tempo(oenv,
                                                                    sr=sr,
                                                                    ac_size=ac_size,
                                                                    std_bpm=std_bpm)
                                
                        for tightness in [1, 10, 50, 100, 500, 1000]:
                            # Evaluate the predictions
                            params = {'aggregate': aggregate.__name__,
                                              'fmax': fmax,
                                              'n_mels': n_mels,
                                              'ac_size': ac_size,
                                              'std_bpm': std_bpm,
                                              'tightness': tightness}
                            
                            _, beats = librosa.beat.beat_track(sr=sr,
                                                                       onset_envelope=oenv,
                                                                       trim=False, 
                                                                       tightness=tightness,
                                                                       bpm=tempo)
                                    
                            est_times = librosa.frames_to_time(beats, sr=sr)
                            scores = mir_eval.beat.evaluate(ref_times, est_times)
                            
                            cont = OrderedDict(index=index)
                            cont.update(params)
                            cont.update(scores)
                            results.append(cont)
    # Blow away the cache
    librosa.cache.clear()
    return pd.DataFrame.from_dict(results, orient='columns')

In [10]:
def analyze_corpus(corpus, max_count=None):
    
    results = None
    for i, idx in enumerate(corpus.index):
        if max_count is not None and i >= max_count:
            break
        new_results = analyze(corpus.loc[[idx]])
        if results is None:
            results = new_results
        else:
            results = pd.concat([results, new_results])
            
    return results

In [11]:
from joblib import Parallel, delayed

In [12]:
def p_analyze_corpus(corpus, n_jobs=3):
    
    results = None
    
    dfunc = delayed(analyze)
    
    results = [Parallel(n_jobs=n_jobs, verbose=10)(dfunc(corpus.loc[[idx]]) for idx in corpus.index)]
    
    return pd.concat(results)

---

In [12]:
smc_data = make_smc_data('/home/bmcfee/data/SMC_Mirex/')

In [13]:
smc_results = analyze_corpus(smc_data)

Processing SMC_001.wav
Processing SMC_002.wav


Processing SMC_003.wav


Processing SMC_004.wav


Processing SMC_005.wav


Processing SMC_006.wav

...


Processing SMC_285.wav


Processing SMC_286.wav


Processing SMC_287.wav


Processing SMC_288.wav


Processing SMC_289.wav

In [15]:
#smc_results.to_json('/home/bmcfee/git/librosa_parameters/smc_beat_results.json', orient='records')

In [13]:
smc_results = pd.read_json('/home/bmcfee/git/librosa_parameters/smc_beat_results.json', orient='records')

In [14]:
smc_scores = smc_results.groupby(['aggregate', 'fmax', 'n_mels', 'ac_size', 'std_bpm', 'tightness']).mean()

In [15]:
smc_scores_std = smc_results.groupby(['aggregate', 'fmax', 'n_mels', 'ac_size', 'std_bpm', 'tightness']).std()

In [16]:
best_igain = smc_scores['Information gain'].argmax()

In [17]:
best_fmeas = smc_scores['F-measure'].argmax()

In [18]:
best_amlt = smc_scores['Any Metric Level Total'].argmax()

In [25]:
smc_scores.loc[('median', 8000.0, 128, 8, 2.0, 100)]

Any Metric Level Continuous          0.172
Any Metric Level Total               0.314
Cemgil                               0.237
Cemgil Best Metric Level             0.305
Correct Metric Level Continuous      0.105
Correct Metric Level Total           0.172
F-measure                            0.353
Goto                                 0.078
Information gain                     0.176
P-score                              0.479
index                              108.000
Name: (median, 8000.0, 128, 8, 2.0, 100), dtype: float64

In [30]:
print smc_scores.loc[best_igain]

Any Metric Level Continuous          0.172
Any Metric Level Total               0.314
Cemgil                               0.237
Cemgil Best Metric Level             0.305
Correct Metric Level Continuous      0.105
Correct Metric Level Total           0.172
F-measure                            0.353
Goto                                 0.078
Information gain                     0.176
P-score                              0.479
index                              108.000
Name: (median, 8000.0, 128, 8, 2.0, 100), dtype: float64


In [31]:
print smc_scores.loc[best_fmeas]

Any Metric Level Continuous          0.079
Any Metric Level Total               0.261
Cemgil                               0.251
Cemgil Best Metric Level             0.322
Correct Metric Level Continuous      0.034
Correct Metric Level Total           0.092
F-measure                            0.368
Goto                                 0.009
Information gain                     0.142
P-score                              0.458
index                              108.000
Name: (mean, 8000.0, 128, 2, 1.0, 10), dtype: float64


In [32]:
print smc_scores.loc[best_amlt]

Any Metric Level Continuous          0.154
Any Metric Level Total               0.334
Cemgil                               0.243
Cemgil Best Metric Level             0.303
Correct Metric Level Continuous      0.097
Correct Metric Level Total           0.173
F-measure                            0.361
Goto                                 0.078
Information gain                     0.174
P-score                              0.493
index                              108.000
Name: (median, 8000.0, 128, 8, 1.0, 50), dtype: float64


---

In [33]:
beatles_data = make_beatles_corpus('/home/bmcfee/data/beatles_iso/')

In [ ]:
beatles_results = p_analyze_corpus(beatles_data.head(5))